# HuggingFace Transformers
This notebook will use the Hugging Face API to interact with different Transformer achitectures.



In [4]:
from transformers import pipeline

# Sentiment analysis

In [5]:
#Create instance of the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 629/629 [00:00<00:00, 315kB/s]
model.safetensors: 100%|██████████| 268M/268M [00:02<00:00, 116MB/s] 
tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 24.0kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.27MB/s]


In [7]:
input = ["You are such a nice person", "You are so lazy, you should really do something good for the world instead"]
classifier(input)

[{'label': 'POSITIVE', 'score': 0.9998427629470825},
 {'label': 'NEGATIVE', 'score': 0.9979380965232849}]

# Text Classification

In [23]:
#Create an instance of the classifier pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [27]:
input = "You should do one more repetition, you can do it!"
candidate_labels = ["politics", "training", "education", "cooking"]

classifier(input, candidate_labels=candidate_labels,)


{'sequence': 'You should do one more repetition, you can do it!',
 'labels': ['training', 'education', 'cooking', 'politics'],
 'scores': [0.8017725348472595,
  0.16816730797290802,
  0.02121736854314804,
  0.008842849172651768]}

In [22]:

oracle = pipeline(model="facebook/bart-large-mnli")

oracle(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5036354064941406,
  0.4787999987602234,
  0.012600099667906761,
  0.0026557904202491045,
  0.0023087533190846443]}

# Text generation

In [13]:
generator = pipeline("text-generation", model='gpt2')

In [28]:
prompt = "Hello, I would like to know the meaning of life. The meaning of life is"
generator(prompt, max_length=142, num_return_sequences=1)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I would like to know the meaning of life. The meaning of life is my responsibility. Your life is of course different from mine. But if you have this responsibility, and you understand your responsibility, why shouldn't we care for you? You are already one of my children. I would try to help you so that you can have a normal life, and I would like for you to go to school and work, and also learn English so that that you could have a better life. I wish that you might see your teacher someday. Now. I would like to talk about some things I noticed over the preceding couple of weeks about my parents, and whether or not I wanted to come back home to my parents and work in the field. I also wanted to say something about my sisters, who are pretty amazing. They were already older than me, and very happy and kind and beautiful, and I could ask them if they love how they are. So, for this particular day, the last time she saw me, she said that she was going to take m

# Named entity recognition

In [31]:
model = pipeline("ner", grouped_entities=True)
text = 'My dog is named Pluto, and he wants to play in the park Pildammarna in Malmö'

model(text)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\cerion\AppData\Roaming\

[{'entity_group': 'PER',
  'score': 0.87922454,
  'word': 'Pluto',
  'start': 16,
  'end': 21},
 {'entity_group': 'LOC',
  'score': 0.9412043,
  'word': 'Pildammarna',
  'start': 56,
  'end': 67},
 {'entity_group': 'LOC',
  'score': 0.9957456,
  'word': 'Malmö',
  'start': 71,
  'end': 76}]